In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import sys
import os
import pandas as pd

In [2]:
from analysis_functions import *

In [15]:
# image = r'C:\Users\Willi\Desktop\134L Images\good_fits\van_Maanen_g.fits'
image = r'C:\Users\admin1\OneDrive\Desktop\134L Images\good_fits\van_Maanen_g.fits'
hdul = fits.open(image)
image = hdul[1].data

# if not os.path.isfile(image):
#     print('File not found')

In [16]:
from photutils.detection import DAOStarFinder
from astropy.stats import sigma_clipped_stats

# Assuming 'image' is a 2D NumPy array of the astronomical image
mean, median, std = sigma_clipped_stats(image, sigma=3.0)

# Initialize DAOStarFinder
daofind = DAOStarFinder(threshold=5.*std, fwhm=3.0)

# Find stars
sources = daofind(image - median)

# Print results
print(sources)

 id     xcentroid          ycentroid      ...    mag      daofind_mag  
--- ------------------ ------------------ ... ---------- --------------
  1 2306.4408154241164  6.259779321721506 ... -0.9937378  -0.0007572887
  2  400.6057567920176 7.1702872270909825 ...  -4.292782   -0.033868663
  3 1991.0681078395646   12.0559356633655 ...  -4.813613 -0.00055278355
  4  808.7954320999415 19.965286804836683 ... -12.029372      -5.367277
  5 1573.9055424720214  27.35116332233888 ... -5.4277506   -0.092916325
  6 1286.5857761003986  36.62858535474353 ... -13.987905      -7.170459
  7 1998.1247807924026  39.84846785034843 ... -2.7234263   -0.008517929
  8   674.168880717519 42.270250345699395 ... -10.257473     -3.5766654
  9   776.946187184268 43.831073603933355 ...  -4.439884     -0.2143856
 10 1306.6600590565452  43.75500959080711 ... -5.2979093   -0.019052442
...                ...                ... ...        ...            ...
403 1894.6812662601913 2354.0199382356213 ...  -4.359886    -0.0

Calculate temperature using color index data

Try with just the imported functions.

In [5]:
# file_root = r'C:\Users\Willi\Desktop\134L Images\good_fits'
file_root = r'C:\Users\admin1\OneDrive\Desktop\134L Images\good_fits'

In [10]:
def calculate_all(file_root, star_name, x_position, y_position, radius):
    star_fluxes = calculate_flux(file_root, star_name, x_position, y_position, radius)[0]
    filter_zeropoints = calculate_flux(file_root, star_name, x_position, y_position, radius)[1]
    apparent_mags = np.zeros(3)
    for i in range(3):
        apparent_mags[i] = apparent_mag(star_fluxes[i], filter_zeropoints[i])

    index_gr = apparent_mags[0] - apparent_mags[1]
    index_ri = apparent_mags[1] - apparent_mags[2]
    index_gi = apparent_mags[0] - apparent_mags[2]
    index_values = [index_gr, index_ri, index_gi]
    print(index_values)

    temperatures = np.zeros(3)
    for i in range(3):
        temperatures[i] = calculate_temperature(index_values[i], color_index_types[i], csv_data)

    average_temp = np.mean(temperatures)
    temp_std = np.std(temperatures)

    return star_fluxes, apparent_mags, temperatures, average_temp, temp_std

In [12]:
calculate_all(file_root, 'WD0621-376_', 1166, 1259, 8)

([5950.871171875, 2245.7225, 545.4123486328124],
 array([12.21520916, 12.70695636, 13.06846054]),
 array([100000., 100000., 100000.]),
 100000.0,
 0.0)

In [13]:
print(relative_to_absolute_magnitude(12.21520916, 76.13))
print(relative_to_absolute_magnitude(12.70695636, 76.13))
print(relative_to_absolute_magnitude(13.06846054, 76.13))

7.807430011033443
8.299177211033442
8.660681391033442


In [11]:
calculate_all(file_root, 'WD0621-376_', 1166, 1259, 8)

[-0.49174718109251714, -0.3615037085030117, -0.8532508895955289]


([42506.223, 16040.875, 3895.8025],
 array([10.08052978, 10.57227696, 10.93378067]),
 array([100000., 100000., 100000.]),
 100000.0,
 0.0)

In [8]:
print(relative_to_absolute_magnitude(10.080, 76.13))
print(relative_to_absolute_magnitude(10.572, 76.13))
print(relative_to_absolute_magnitude(10.933, 76.13))

5.672220851033442
6.164220851033441
6.525220851033442


In [6]:
calculate_all(file_root, 'LHS_253_', 1251, 1175, 8)

([58707.164, 35686.727, 10611.516],
 array([9.64278643, 9.62425715, 9.71333189]),
 array([10500., 14900., 11700.]),
 12366.666666666666,
 1857.1184369578827)

In [6]:
print(relative_to_absolute_magnitude(9.6427, 8.52))
print(relative_to_absolute_magnitude(9.6242, 8.52))
print(relative_to_absolute_magnitude(9.713, 8.52))

9.9905020261665
9.9720020261665
10.060802026166499


In [6]:
calculate_all(file_root, 'LAWD_8_', 1208, 1216, 8)

([944.6901406249999, 551.456390625, 167.00490624999998],
 array([14.18279463, 14.231414  , 14.37469643]),
 array([11700., 19700., 13500.]),
 14966.666666666666,
 3426.6926069057063)

In [9]:
print(relative_to_absolute_magnitude(14.18279463, 16.761))
print(relative_to_absolute_magnitude(14.231414, 16.761))
print(relative_to_absolute_magnitude(14.37469643, 16.761))

13.061294999600866
13.109914369600865
13.253196799600866


In [10]:
calculate_all(file_root, 'van_Maanen_', 1189, 1173, 8)

([2985.94625, 2157.8111132812496, 894.7300195312498],
 array([12.96293269, 12.69025376, 12.65932917]),
 array([7600., 9900., 8300.]),
 8600.0,
 962.6352718795767)

In [11]:
print(relative_to_absolute_magnitude(12.96293269, 4.32))
print(relative_to_absolute_magnitude(12.69025376, 4.32))
print(relative_to_absolute_magnitude(12.65932917, 4.32))

14.78551395592544
14.512835025925439
14.481910435925439


In [5]:
calculate_all(file_root, 'van_Maanen_', 1189, 1173, 8)

([170625.5, 123303.49, 51127.43],
 array([8.57052829, 8.29784911, 8.26692438]),
 array([7600., 9900., 8300.]),
 8600.0,
 962.6352718795767)

In [4]:
print(relative_to_absolute_magnitude(8.5705282, 4.32))
print(relative_to_absolute_magnitude(8.2978, 4.32))
print(relative_to_absolute_magnitude(8.2669, 4.32))

10.39310946592544
10.12038126592544
10.08948126592544


In [14]:
calculate_all(file_root, 'omi02EriB_', 1213, 1233, 7)

([7335.29453125, 10858.06298828125, 14840.624804687499],
 array([12.03485102, 11.01811695,  9.49676254]),
 array([4300., 2200., 3200.]),
 3233.3333333333335,
 857.6453553512405)

In [15]:
print(relative_to_absolute_magnitude(12.03485102, 5.008))
print(relative_to_absolute_magnitude(11.01811695, 5.008))
print(relative_to_absolute_magnitude(9.49676254, 5.008))

13.536529418988133
12.519795348988133
10.998440938988134


In [8]:
calculate_all(file_root, 'omi02EriB_', 1213, 1233, 7)

([10478.992, 15511.519, 21200.893],
 array([11.64759626, 10.63086215,  9.10950793]),
 array([4300., 2200., 3200.]),
 3233.3333333333335,
 857.6453553512405)

In [7]:
print(relative_to_absolute_magnitude(11.647, 5.008))
print(relative_to_absolute_magnitude(10.630, 5.008))
print(relative_to_absolute_magnitude(9.10950, 5.008))

13.148678398988134
12.131678398988134
10.611178398988134
